<a href="https://colab.research.google.com/github/GayathriKodipaka/IBM-hackathon/blob/main/ibm_granite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio pandas numpy matplotlib seaborn plotly

In [ ]:
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import json
import re
import random
from collections import Counter, defaultdict
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import threading
import time

# Enhanced Government Platform with IBM Granite AI Integration
class GovernmentPlatform:
    def __init__(self):
        self.supported_languages = ["English", "Hindi", "Telugu", "Tamil", "Bengali", "Marathi"]
        self.query_categories = ["Healthcare", "Education", "Infrastructure", "Employment", "Welfare", "Legal", "Other"]
        self.setup_theme()
        self.init_mock_data()
        self.init_granite_model()
        self.setup_government_knowledge_base()

    def setup_government_knowledge_base(self):
        """Setup comprehensive government knowledge base for better responses"""
        self.knowledge_base = {
            "passport": {
                "process": "Apply online at passportindia.gov.in → Fill application → Book appointment → Visit PSK → Submit documents → Verification → Passport delivery",
                "documents": "Proof of identity (Aadhaar/Voter ID), Proof of address, Birth certificate, Photographs",
                "fees": "Fresh passport: ₹1,500 (36 pages), ₹2,000 (60 pages). Tatkal: Additional ₹2,000",
                "timeline": "Normal: 30-45 days, Tatkal: 1-3 days",
                "offices": "Passport Seva Kendra (PSK) or Post Office Passport Seva Kendra (POPSK)"
            },
            "ration_card": {
                "process": "Apply online on state food portal → Submit application → Document verification → Ration card issuance",
                "documents": "Identity proof, Address proof, Income certificate, Family photograph",
                "types": "AAY (Antyodaya), BPL (Below Poverty Line), APL (Above Poverty Line)",
                "benefits": "Subsidized food grains, LPG subsidy, other welfare schemes",
                "timeline": "15-30 days after verification"
            },
            "birth_certificate": {
                "process": "Apply at municipal corporation/registrar office → Submit application → Document verification → Certificate issuance",
                "documents": "Hospital discharge certificate, Identity proof of parents, Address proof",
                "timeline": "Immediate for births registered within 21 days, 7-15 days for delayed registration",
                "fees": "₹5-50 depending on state",
                "online": "Most states offer online application through state portals"
            },
            "pension": {
                "types": "Old Age Pension (60+ years), Widow Pension, Disability Pension",
                "eligibility": "Age criteria, Income limits, Residency proof",
                "amount": "₹200-1,000 per month (varies by state)",
                "documents": "Age proof, Income certificate, Bank details, Aadhaar",
                "application": "Social Welfare Department or online through state portals"
            },
            "healthcare": {
                "ayushman_bharat": "Health insurance up to ₹5 lakh per family per year",
                "eligibility": "SECC database families, rural and urban poor",
                "services": "Primary health centers, Community health centers, Government hospitals",
                "emergency": "108 ambulance service, Emergency helplines",
                "schemes": "Pradhan Mantri Jan Arogya Yojana (PM-JAY)"
            },
            "education": {
                "scholarships": "Merit-based and need-based scholarships for students",
                "schemes": "Mid-day meals, Free textbooks, Sarva Shiksha Abhiyan",
                "higher_education": "Central and state universities, Technical education",
                "online": "SWAYAM, e-pathshala, Digital India initiatives"
            },
            "employment": {
                "mgnrega": "100 days guaranteed employment for rural households",
                "skill_development": "Pradhan Mantri Kaushal Vikas Yojana (PMKVY)",
                "job_portals": "National Career Service (NCS), Employment exchanges",
                "self_employment": "Mudra loans, Start-up India initiatives"
            },
            "legal": {
                "legal_aid": "Free legal aid for eligible citizens through DLSA",
                "consumer_rights": "Consumer forums for grievance redressal",
                "helplines": "Legal aid helplines, Women helplines",
                "procedures": "Filing FIR, Bail procedures, Court processes"
            }
        }

        self.common_queries = {
            "how to apply": "To apply for {service}, you need to {process}. Required documents include {documents}.",
            "documents required": "For {service}, you need: {documents}",
            "fees": "The fees for {service} are: {fees}",
            "timeline": "The processing time for {service} is: {timeline}",
            "eligibility": "The eligibility criteria for {service} are: {eligibility}",
            "status check": "You can check the status of your {service} application online or by contacting the relevant office.",
            "helpline": "For {service} related queries, you can contact the helpline or visit the nearest office."
        }


    def init_granite_model(self):
        """Initialize IBM Granite 3.3 2B Instruct model with enhanced configuration"""
        try:
            print("Loading IBM Granite 3.3 2B Instruct model...")
            self.model_name = "ibm-granite/granite-3.3-2b-instruct"

            # Load tokenizer and model with better configuration
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, trust_remote_code=True)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                device_map="auto" if torch.cuda.is_available() else None,
                trust_remote_code=True,
                low_cpu_mem_usage=True
            )

            # Create enhanced text generation pipeline
            self.text_generator = pipeline(
                "text-generation",
                model=self.model,
                tokenizer=self.tokenizer,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                device_map="auto" if torch.cuda.is_available() else None,
                max_new_tokens=400,
                do_sample=True,
                temperature=0.6,
                top_p=0.85,
                top_k=50,
                repetition_penalty=1.1,
                pad_token_id=self.tokenizer.eos_token_id
            )

            # Enhanced government-specific context
            self.government_context = """You are an expert AI assistant for the Government of India's Citizen Engagement Platform. You provide accurate, helpful, and official information about government services, procedures, and policies.

Your expertise includes:
- Government services (passport, ration card, birth certificate, etc.)
- Welfare schemes and benefits (pension, healthcare, education)
- Legal procedures and citizen rights
- Employment opportunities and skill development
- Infrastructure and civic issues
- Digital government initiatives

Guidelines:
- Provide step-by-step procedures when explaining processes
- Mention required documents, fees, and timelines
- Include relevant government websites and helplines
- Be empathetic and understanding of citizen concerns
- Use simple, clear language
- Offer alternatives when possible
- Always maintain professional, helpful tone

If you don't know specific current information, guide users to appropriate official sources."""

            self.model_loaded = True
            print("✅ IBM Granite model loaded successfully!")

        except Exception as e:
            print(f"❌ Error loading Granite model: {e}")
            print("🔄 Falling back to enhanced rule-based responses...")
            self.model_loaded = False
            self.setup_enhanced_fallback_responses()

    def setup_enhanced_fallback_responses(self):
        """Setup comprehensive fallback responses with government knowledge"""
        self.fallback_responses = {
            "passport": f"""For passport services:
📋 *Process*: {self.knowledge_base['passport']['process']}
📄 *Documents*: {self.knowledge_base['passport']['documents']}
💰 *Fees*: {self.knowledge_base['passport']['fees']}
⏱ *Timeline*: {self.knowledge_base['passport']['timeline']}
🏢 *Office*: {self.knowledge_base['passport']['offices']}
🌐 *Website*: passportindia.gov.in""",

            "ration card": f"""For ration card application:
📋 *Process*: {self.knowledge_base['ration_card']['process']}
📄 *Documents*: {self.knowledge_base['ration_card']['documents']}
📊 *Types*: {self.knowledge_base['ration_card']['types']}
💰 *Benefits*: {self.knowledge_base['ration_card']['benefits']}
⏱ *Timeline*: {self.knowledge_base['ration_card']['timeline']}""",

            "birth certificate": f"""For birth certificate:
📋 *Process*: {self.knowledge_base['birth_certificate']['process']}
📄 *Documents*: {self.knowledge_base['birth_certificate']['documents']}
⏱ *Timeline*: {self.knowledge_base['birth_certificate']['timeline']}
💰 *Fees*: {self.knowledge_base['birth_certificate']['fees']}
🌐 *Online*: {self.knowledge_base['birth_certificate']['online']}""",

            "pension": f"""For pension schemes:
📊 *Types*: {self.knowledge_base['pension']['types']}
✅ *Eligibility*: {self.knowledge_base['pension']['eligibility']}
💰 *Amount*: {self.knowledge_base['pension']['amount']}
📄 *Documents*: {self.knowledge_base['pension']['documents']}
📋 *Application*: {self.knowledge_base['pension']['application']}""",

            "healthcare": f"""For healthcare services:
🏥 *Ayushman Bharat*: {self.knowledge_base['healthcare']['ayushman_bharat']}
✅ *Eligibility*: {self.knowledge_base['healthcare']['eligibility']}
🏥 *Services*: {self.knowledge_base['healthcare']['services']}
🚨 *Emergency*: {self.knowledge_base['healthcare']['emergency']}
📋 *Schemes*: {self.knowledge_base['healthcare']['schemes']}""",

            "education": f"""For education services:
🎓 *Scholarships*: {self.knowledge_base['education']['scholarships']}
📚 *Schemes*: {self.knowledge_base['education']['schemes']}
🏫 *Higher Education*: {self.knowledge_base['education']['higher_education']}
💻 *Online*: {self.knowledge_base['education']['online']}""",

            "employment": f"""For employment opportunities:
💼 *MGNREGA*: {self.knowledge_base['employment']['mgnrega']}
🎯 *Skill Development*: {self.knowledge_base['employment']['skill_development']}
🌐 *Job Portals*: {self.knowledge_base['employment']['job_portals']}
💰 *Self Employment*: {self.knowledge_base['employment']['self_employment']}""",

            "legal": f"""For legal assistance:
⚖ *Legal Aid*: {self.knowledge_base['legal']['legal_aid']}
👥 *Consumer Rights*: {self.knowledge_base['legal']['consumer_rights']}
📞 *Helplines*: {self.knowledge_base['legal']['helplines']}
📋 *Procedures*: {self.knowledge_base['legal']['procedures']}"""
        }

    def setup_theme(self):
        """Setup government-themed interface"""
        self.theme = gr.themes.Soft(
            primary_hue="blue",
            secondary_hue="indigo",
            neutral_hue="slate"
        ).set(
            body_background_fill="linear-gradient(135deg, #667eea 0%, #764ba2 100%)",
            block_background_fill="white",
            block_border_color="#e5e7eb"
        )

    def init_mock_data(self):
        """Initialize mock data for demonstration"""
        self.citizens_data = pd.DataFrame({
            'citizen_id': range(1, 1001),
            'queries_count': np.random.poisson(3, 1000),
            'satisfaction_score': np.random.uniform(1, 5, 1000),
            'language': np.random.choice(self.supported_languages, 1000),
            'category': np.random.choice(self.query_categories, 1000),
            'resolution_time': np.random.exponential(2, 1000)
        })

        self.queries_data = []
        for i in range(100):
            self.queries_data.append({
                'id': f"Q{i+1:03d}",
                'citizen_id': f"C{random.randint(1, 1000):04d}",
                'query': f"Sample query about {random.choice(self.query_categories).lower()}",
                'category': random.choice(self.query_categories),
                'sentiment': random.choice(['Positive', 'Neutral', 'Negative']),
                'status': random.choice(['Open', 'In Progress', 'Resolved', 'Closed']),
                'priority': random.choice(['Low', 'Medium', 'High', 'Critical']),
                'timestamp': datetime.now() - timedelta(days=random.randint(0, 30))
            })
    def extract_intent_and_entities(self, message):
        """Extract intent and entities from user message"""
        message_lower = message.lower()

        # Intent detection
        intent = "general_inquiry"
        if any(word in message_lower for word in ["how to", "how do", "steps", "process", "procedure"]):
            intent = "process_inquiry"
        elif any(word in message_lower for word in ["documents", "papers", "required", "need"]):
            intent = "document_inquiry"
        elif any(word in message_lower for word in ["fees", "cost", "price", "charges"]):
            intent = "fee_inquiry"
        elif any(word in message_lower for word in ["time", "timeline", "duration", "when"]):
            intent = "timeline_inquiry"
        elif any(word in message_lower for word in ["eligibility", "eligible", "qualify"]):
            intent = "eligibility_inquiry"
        elif any(word in message_lower for word in ["status", "track", "check"]):
            intent = "status_inquiry"
        elif any(word in message_lower for word in ["help", "helpline", "contact", "support"]):
            intent = "contact_inquiry"

        # Entity extraction (service type)
        entities = []
        for service in self.knowledge_base.keys():
            if service in message_lower or service.replace("_", " ") in message_lower:
                entities.append(service)

        return intent, entities

    def generate_structured_response(self, intent, entities, message):
        """Generate structured response based on intent and entities"""
        if not entities:
            return self.get_general_response(message)

        service = entities[0]  # Use first detected service

        if service not in self.knowledge_base:
            return self.get_general_response(message)

        service_info = self.knowledge_base[service]

        response = f"{service.replace('_', ' ').title()} Information:\n\n"

        if intent == "process_inquiry":
            if "process" in service_info:
                response += f"📋 *Process*: {service_info['process']}\n\n"
            if "documents" in service_info:
                response += f"📄 *Required Documents*: {service_info['documents']}\n\n"
        elif intent == "document_inquiry":
            if "documents" in service_info:
                response += f"📄 *Required Documents*: {service_info['documents']}\n\n"
        elif intent == "fee_inquiry":
            if "fees" in service_info:
                response += f"💰 *Fees*: {service_info['fees']}\n\n"
        elif intent == "timeline_inquiry":
            if "timeline" in service_info:
                response += f"⏱ *Timeline*: {service_info['timeline']}\n\n"
        elif intent == "eligibility_inquiry":
            if "eligibility" in service_info:
                response += f"✅ *Eligibility*: {service_info['eligibility']}\n\n"
        else:
            # Provide comprehensive information
            for key, value in service_info.items():
                if key == "process":
                    response += f"📋 *Process*: {value}\n\n"
                elif key == "documents":
                    response += f"📄 *Documents*: {value}\n\n"
                elif key == "fees":
                    response += f"💰 *Fees*: {value}\n\n"
                elif key == "timeline":
                    response += f"⏱ *Timeline*: {value}\n\n"
                elif key == "eligibility":
                    response += f"✅ *Eligibility*: {value}\n\n"
                elif key == "types":
                    response += f"📊 *Types*: {value}\n\n"
                elif key == "benefits":
                    response += f"🎁 *Benefits*: {value}\n\n"

        response += "\n💡 *Need more help?* Feel free to ask specific questions or contact the relevant government office."

        return response

    def get_general_response(self, message):
        """Get general response for unmatched queries"""
        return """I'm here to help you with government services and procedures. I can assist you with:

🏛 *Government Services:*
- Passport applications and renewals
- Ration card applications
- Birth, death, and marriage certificates
- Pension schemes and benefits

🏥 *Healthcare Services:*
- Ayushman Bharat health insurance
- Government hospitals and clinics
- Health schemes and programs

🎓 *Education Services:*
- Scholarships and financial aid
- School admissions and transfers
- Higher education opportunities

💼 *Employment Services:*
- MGNREGA rural employment
- Skill development programs
- Government job opportunities

⚖ *Legal Services:*
- Legal aid and assistance
- Consumer rights and complaints
- Court procedures and documentation

Please ask me specific questions about any of these services, and I'll provide detailed information including required documents, procedures, fees, and timelines."""

    def analyze_sentiment(self, text):
        """Enhanced sentiment analysis"""
        # Expanded sentiment keywords
        positive_words = ['good', 'great', 'excellent', 'satisfied', 'happy', 'pleased', 'wonderful',
                         'amazing', 'perfect', 'helpful', 'thank', 'thanks', 'appreciate', 'love', 'awesome']
        negative_words = ['bad', 'terrible', 'awful', 'unsatisfied', 'angry', 'frustrated', 'horrible',
                         'worst', 'disappointed', 'annoyed', 'hate', 'disgusted', 'upset', 'concern', 'problem']
        neutral_words = ['okay', 'fine', 'average', 'normal', 'standard', 'regular']

        text_lower = text.lower()

        # Count sentiment indicators
        positive_count = sum(1 for word in positive_words if word in text_lower)
        negative_count = sum(1 for word in negative_words if word in text_lower)
        neutral_count = sum(1 for word in neutral_words if word in text_lower)

        # Determine sentiment with confidence
        if positive_count > negative_count and positive_count > neutral_count:
            sentiment = "😊 Positive"
            confidence = min(90, 60 + (positive_count * 10))
        elif negative_count > positive_count and negative_count > neutral_count:
            sentiment = "😞 Negative"
            confidence = min(90, 60 + (negative_count * 10))
        elif neutral_count > 0:
            sentiment = "😐 Neutral"
            confidence = min(80, 50 + (neutral_count * 10))
        else:
            sentiment = "😐 Neutral"
            confidence = 50

        return sentiment, confidence

    def generate_granite_response(self, user_message, chat_history):
        """Enhanced response generation using IBM Granite model"""
        try:
            # Extract intent and entities first
            intent, entities = self.extract_intent_and_entities(user_message)

            # Build conversation context
            conversation_context = self.government_context + "\n\n"

            # Add relevant knowledge base information
            if entities:
                for entity in entities[:2]:  # Limit to first 2 entities
                    if entity in self.knowledge_base:
                        conversation_context += f"Knowledge about {entity}:\n"
                        for key, value in self.knowledge_base[entity].items():
                            conversation_context += f"- {key}: {value}\n"
                        conversation_context += "\n"

            # Add recent chat history for context
            if chat_history:
                conversation_context += "Recent conversation:\n"
                for human_msg, ai_msg in chat_history[-2:]:  # Last 2 exchanges
                    conversation_context += f"Human: {human_msg}\nAssistant: {ai_msg}\n\n"

            # Create enhanced prompt
            prompt = f"""{conversation_context}

Current user query: {user_message}
Intent: {intent}
Entities: {entities}

Please provide a helpful, accurate, and detailed response about the government service or procedure. Include specific steps, required documents, fees, timelines, and relevant contact information where applicable.

Response:"""

            # Generate response with better parameters
            response = self.text_generator(
                prompt,
                max_new_tokens=350,
                do_sample=True,
                temperature=0.6,
                top_p=0.85,
                top_k=50,
                repetition_penalty=1.1,
                pad_token_id=self.tokenizer.eos_token_id
            )

            # Extract and clean the response
            generated_text = response[0]['generated_text']
            assistant_response = generated_text.split("Response:")[-1].strip()

            # Clean up the response
            assistant_response = assistant_response.split("Human:")[0].strip()
            assistant_response = assistant_response.split("User:")[0].strip()

            # Remove any repeated phrases
            lines = assistant_response.split('\n')
            cleaned_lines = []
            for line in lines:
                if line.strip() and line.strip() not in cleaned_lines:
                    cleaned_lines.append(line.strip())

            assistant_response = '\n'.join(cleaned_lines)

            # Ensure minimum quality
            if len(assistant_response) < 50 or not assistant_response:
                return self.generate_structured_response(intent, entities, user_message)

            # Add helpful formatting
            if not assistant_response.startswith(('', '🏛', '📋', '💡')):
                assistant_response = f"🏛 *Government Service Information*\n\n{assistant_response}"

            return assistant_response

        except Exception as e:
            print(f"Error generating Granite response: {e}")
            return self.generate_structured_response(intent, entities, user_message)


    def get_fallback_response(self, message):
        """Get fallback response when AI model fails"""
        message_lower = message.lower()

        # Check for keywords in fallback responses
        for keyword, response in self.fallback_responses.items():
            if keyword in message_lower:
                return response

        # Default response
        return "I'm here to help you with government services and procedures. Could you please provide more specific details about what you need assistance with? I can help with passport applications, ration cards, birth certificates, pension schemes, healthcare services, education policies, employment opportunities, legal aid, and infrastructure issues."

    def chatbot_response(self, message, chat_history):
        """Enhanced chatbot response with improved AI integration"""
        if not message.strip():
            return "", chat_history

        # Generate response
        if self.model_loaded:
            try:
                response = self.generate_granite_response(message, chat_history)
            except Exception as e:
                print(f"Granite model error: {e}")
                response = self.get_fallback_response(message)
        else:
            response = self.get_fallback_response(message)

        # Analyze sentiment with confidence
        sentiment, confidence = self.analyze_sentiment(message)

        # Add enhanced metadata
        metadata = f"""
---
📊 *Analysis*: {sentiment} (Confidence: {confidence}%)
⏰ *Response Time*: {datetime.now().strftime('%H:%M:%S')}
🤖 *AI Model*: {'IBM Granite 3.3-2B' if self.model_loaded else 'Enhanced Fallback System'}
💬 *Query Type*: Government Service Inquiry
"""

        # Combine response with metadata
        final_response = f"{response}\n\n{metadata}"

        # Add to chat history
        chat_history.append([message, final_response])

        return "", chat_history

    # Keep all other methods unchanged from the original implementation

    def process_citizen_query(self, query_text, language, voice_input, category):
        """Process citizen queries with AI enhancement"""
        if not query_text and not voice_input:
            return "Please provide a query either as text or voice input."

        processed_query = query_text if query_text else "[Voice input transcribed]"

        # Enhanced AI analysis
        if self.model_loaded and query_text:
            try:
                intent, entities = self.extract_intent_and_entities(query_text)

                analysis_prompt = f"""
                Analyze this citizen query:
                Query: {query_text}
                Category: {category}
                Intent: {intent}
                Entities: {entities}

                Provide:
                1. Priority level (Low/Medium/High/Critical)
                2. Estimated resolution time
                3. Required department
                4. Recommended action
                """

                ai_analysis = self.text_generator(
                    analysis_prompt,
                    max_new_tokens=150,
                    temperature=0.3
                )[0]['generated_text'].split("Provide:")[-1].strip()

            except:
                ai_analysis = "Standard processing with intent analysis applied"
        else:
            ai_analysis = "Standard processing applied"

        response = {
            "query_id": f"Q{random.randint(1000, 9999)}",
            "original_language": language,
            "processed_query": processed_query,
            "category": category,
            "intent_analysis": "Detected intent and entities for better routing",
            "ai_analysis": ai_analysis,
            "estimated_resolution_time": f"{random.randint(1, 7)} days",
            "status": "Received and processed with enhanced AI analysis",
            "reference_number": f"REF{datetime.now().strftime('%Y%m%d%H%M%S')}",
            "ai_model": "IBM Granite 3.3-2B Enhanced" if self.model_loaded else "Structured Fallback System",
            "next_steps": "You will receive updates via SMS and email. Track status online using reference number."
        }

        return json.dumps(response, indent=2)



    def detect_misinformation(self, text_content, source_url):
        """Enhanced misinformation detection with AI"""
        if not text_content:
            return "Please provide content to analyze"

        # Enhanced risk assessment
        risk_keywords = ['fake', 'hoax', 'conspiracy', 'unverified', 'rumor', 'false', 'misleading', 'scam', 'fraud']
        credibility_indicators = ['official', 'government', 'verified', 'authentic', 'documented', 'certified']

        text_lower = text_content.lower()
        risk_score = sum(1 for keyword in risk_keywords if keyword in text_lower)
        credibility_score = sum(1 for indicator in credibility_indicators if indicator in text_lower)

        # Enhanced AI analysis
        if self.model_loaded:
            try:
                verification_prompt = f"""
                Analyze this content for misinformation:

                Content: {text_content[:800]}...
                Source: {source_url}

                Evaluate:
                1. Factual accuracy indicators
                2. Source credibility signals
                3. Emotional manipulation tactics
                4. Verification requirements
                5. Risk assessment

                Analysis:
                """

                ai_verification = self.text_generator(
                    verification_prompt,
                    max_new_tokens=250,
                    temperature=0.3
                )[0]['generated_text'].split("Analysis:")[-1].strip()

            except:
                ai_verification = "Enhanced rule-based analysis applied"
        else:
            ai_verification = "Rule-based analysis with keyword detection"

        # Calculate enhanced credibility score
        base_score = max(0, 100 - (risk_score * 12) + (credibility_score * 8))
        final_score = max(0, min(100, base_score + random.randint(-15, 15)))

        analysis = {
            "content_analysis": {
                "length": len(text_content),
                "risk_keywords_detected": risk_score,
                "credibility_indicators": credibility_score,
                "language_analysis": "Formal" if any(word in text_lower for word in ['government', 'official', 'ministry']) else "Informal"
            },
            "credibility_assessment": {
                "score": f"{final_score}%",
                "classification": "High Risk" if final_score < 40 else "Medium Risk" if final_score < 70 else "Low Risk",
                "confidence_level": "High" if abs(final_score - 50) > 30 else "Medium"
            },
            "ai_analysis": ai_verification,
            "verification_steps": [
                "✅ Cross-reference with official government websites",
                "✅ Check PIB (Press Information Bureau) fact-checks",
                "✅ Verify with ministry/department websites",
                "✅ Consult official social media accounts",
                "✅ Contact relevant government helplines"
            ] if final_score < 70 else [
                "✅ Content appears credible",
                "✅ Consider cross-referencing for complete accuracy"
            ],
            "recommendations": {
                "citizen_action": "Verify before sharing" if final_score < 70 else "Appears reliable",
                "official_sources": ["pib.gov.in", "mygov.in", "india.gov.in"],
                "fact_check_resources": ["PIB Fact Check", "MyGov Corona Helpdesk"]
            },
            "metadata": {
                "analysis_timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "ai_model": "IBM Granite 3.3-2B Enhanced" if self.model_loaded else "Enhanced Rule-based System",
                "version": "2.0"
            }
        }
    def get_summary(self):
        return {
            'total_queries': len(self.citizens_data['queries_count']),
            'avg_satisfaction': round(np.mean(self.citizens_data['satisfaction_score']), 2),
            'avg_resolution_time': round(np.mean(self.citizens_data['resolution_time']), 2)
        }

    def filter_by_language(self, lang):
        mask = np.array(self.citizens_data['language']) == lang
        return {key: np.array(val)[mask] for key, val in self.citizens_data.items()}

    def generate_admin_dashboard(self):
        """Generate real-time dashboard analytics"""
        dates = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D')
        query_volume = np.random.poisson(50, len(dates))

        fig1 = px.line(x=dates, y=query_volume, title="Daily Query Volume")
        fig1.update_layout(xaxis_title="Date", yaxis_title="Number of Queries")

        category_counts = Counter(self.citizens_data['category'])
        fig2 = px.pie(values=list(category_counts.values()),
                     names=list(category_counts.keys()),
                     title="Query Distribution by Category")

        fig3 = px.histogram(self.citizens_data, x='satisfaction_score',
                           title="Citizen Satisfaction Distribution",
                           nbins=20)

        language_counts = Counter(self.citizens_data['language'])
        fig4 = px.bar(x=list(language_counts.keys()),
                     y=list(language_counts.values()),
                     title="Platform Usage by Language")

        return fig1, fig2, fig3, fig4

    def staff_query_management(self, staff_role, filter_category, filter_status):
        """Government staff query management interface"""
        filtered_queries = self.queries_data.copy()

        if filter_category != "All":
            filtered_queries = [q for q in filtered_queries if q['category'] == filter_category]

        if filter_status != "All":
            filtered_queries = [q for q in filtered_queries if q['status'] == filter_status]

        df = pd.DataFrame(filtered_queries)

        summary = {
            "total_queries": len(filtered_queries),
            "open_queries": len([q for q in filtered_queries if q['status'] == 'Open']),
            "resolved_queries": len([q for q in filtered_queries if q['status'] == 'Resolved']),
            "average_resolution_time": f"{np.mean([q['timestamp'].day for q in filtered_queries]):.1f} days",
            "staff_role": staff_role,
            "ai_assistance": "IBM Granite 3.3-2B Active" if self.model_loaded else "Basic System"
        }

        return df.head(20), json.dumps(summary, indent=2)

    def generate_analytics_report(self, report_type, date_range):
        """Generate comprehensive analytics reports"""
        if report_type == "Performance Metrics":
            metrics = {
                "total_citizens_served": len(self.citizens_data),
                "average_satisfaction_score": f"{self.citizens_data['satisfaction_score'].mean():.2f}/5.0",
                "total_queries_processed": len(self.queries_data),
                "resolution_rate": f"{random.randint(75, 95)}%",
                "average_response_time": f"{random.randint(2, 8)} hours",
                "most_popular_service": random.choice(self.query_categories),
                "peak_usage_time": "10:00 AM - 2:00 PM",
                "mobile_app_usage": f"{random.randint(60, 80)}%",
                "ai_model_status": "IBM Granite 3.3-2B Active" if self.model_loaded else "Fallback System"
            }

        elif report_type == "Trend Analysis":
            metrics = {
                "query_growth_rate": f"+{random.randint(5, 25)}% monthly",
                "satisfaction_trend": "Improving (+0.3 points)",
                "digital_adoption_rate": f"{random.randint(70, 90)}%",
                "language_preference_shift": "Increasing vernacular usage",
                "service_demand_forecast": "Healthcare queries expected to increase by 15%",
                "seasonal_patterns": "Higher queries during festival seasons",
                "user_retention_rate": f"{random.randint(80, 95)}%",
                "ai_enhancement_impact": "25% improvement in response quality" if self.model_loaded else "Baseline performance"
            }

        else:  # Predictive Insights
            metrics = {
                "predicted_query_volume": f"{random.randint(1000, 5000)} next month",
                "resource_optimization": "Deploy 2 additional staff in Healthcare dept",
                "citizen_satisfaction_prediction": "Expected to reach 4.5/5.0",
                "service_expansion_recommendation": "Add multi-modal transport services",
                "infrastructure_needs": "Upgrade server capacity by 30%",
                "staff_training_priority": "AI-assisted query resolution",
                "budget_allocation_suggestion": "45% tech/AI, 30% staff, 25% infrastructure"
            }

        return json.dumps(metrics, indent=2)

    def send_notifications(self, notification_type, target_audience, message, channels):
        """Send notifications and alerts to citizens"""
        simulation = {
            "notification_id": f"NOT_{datetime.now().strftime('%Y%m%d%H%M%S')}",
            "type": notification_type,
            "target_audience": target_audience,
            "message": message,
            "channels": channels,
            "estimated_reach": random.randint(1000, 50000),
            "delivery_status": "Scheduled for immediate delivery",
            "cost_estimate": f"₹{random.randint(500, 5000)}",
            "delivery_time": f"{random.randint(5, 30)} minutes",
            "success_rate_prediction": f"{random.randint(85, 98)}%"
        }

        return json.dumps(simulation, indent=2)

    def authenticate_user(self, username, role, access_level):
        """Simulate user authentication and role-based access"""
        if not username:
            return "Please provide username"

        auth_result = {
            "username": username,
            "role": role,
            "access_level": access_level,
            "login_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "session_id": f"SES_{random.randint(100000, 999999)}",
            "permissions": {
                "view_dashboard": role in ["Admin", "Manager", "Staff"],
                "manage_queries": role in ["Admin", "Manager", "Staff"],
                "access_analytics": role in ["Admin", "Manager"],
                "system_administration": role == "Admin",
                "citizen_data_access": access_level in ["Full", "Partial"],
                "ai_model_access": self.model_loaded
            },
            "last_login": (datetime.now() - timedelta(days=random.randint(1, 7))).strftime("%Y-%m-%d"),
            "failed_attempts": 0,
            "account_status": "Active",
            "ai_features": "IBM Granite 3.3-2B Enabled" if self.model_loaded else "Basic Features"
        }

        return json.dumps(auth_result, indent=2)

    def create_interface(self):
        """Create the comprehensive government platform interface"""

        with gr.Blocks(theme=self.theme, title="Government Citizen Engagement Platform") as interface:

            # Header
            gr.Markdown(f"""
            # 🏛️ Government Citizen Engagement Platform
            ### IBM Hackathon 2024 - Digital Governance Solution

            **AI Enhancement Status:** {'✅ IBM Granite 3.3-2B Active' if self.model_loaded else '⚠️ Fallback System Active'}

            Comprehensive platform for citizen services, query management, and government analytics
            """)

            with gr.Tabs():
                # Module 1: Citizen Interface
                with gr.TabItem("👥 Citizen Portal"):
                    gr.Markdown("### Submit Your Query - Voice & Multi-language Support")

                    with gr.Row():
                        with gr.Column():
                            citizen_query = gr.Textbox(
                                label="Your Query",
                                placeholder="Describe your issue or question...",
                                lines=4
                            )
                            language_select = gr.Dropdown(
                                choices=self.supported_languages,
                                label="Preferred Language",
                                value="English"
                            )
                            voice_input = gr.Audio(
                                label="Voice Input (Optional)",
                                type="filepath"
                            )
                            query_category = gr.Dropdown(
                                choices=self.query_categories,
                                label="Query Category",
                                value="Other"
                            )
                            submit_query_btn = gr.Button("📤 Submit Query", variant="primary")

                        with gr.Column():
                            query_response = gr.Textbox(
                                label="Query Processing Result",
                                lines=12,
                                show_copy_button=True
                            )

                # Module 2: Enhanced AI Chatbot
                with gr.TabItem("🤖 AI Assistant (IBM Granite)"):
                    gr.Markdown(f"""
                    ### AI-Powered Government Assistant
                    **Powered by:** {'IBM Granite 3.3-2B Instruct Model' if self.model_loaded else 'Fallback Response System'}

                    Get instant help with government services, procedures, and information.
                    """)

                    chatbot = gr.Chatbot(
                        label="Government AI Assistant",
                        height=400,
                        placeholder="Ask me about government services, procedures, or any questions you have!"
                    )
                    with gr.Row():
                        chat_input = gr.Textbox(
                            label="Ask me anything...",
                            placeholder="Type your question here (e.g., 'How to apply for passport?')",
                            scale=4
                        )
                        chat_submit = gr.Button("Send", variant="primary", scale=1)

                    gr.Examples(
                        examples=[
                            "How do I apply for a ration card?",
                            "What documents are needed for passport application?",
                            "Tell me about pension scheme eligibility",
                            "How to get a birth certificate?",
                            "What are the steps for starting a business?",
                            "How can I access healthcare services?",
                            "What employment opportunities are available?",
                            "How do I file a legal complaint?"
                        ],
                        inputs=chat_input
                    )

                # Module 3: Enhanced Misinformation Detection
                with gr.TabItem("🔍 AI Content Verification"):
                    gr.Markdown(f"""
                    ### AI-Powered Content Verification System
                    **Enhanced with:** {'IBM Granite 3.3-2B Analysis' if self.model_loaded else 'Rule-based Detection'}
                    """)

                    with gr.Row():
                        with gr.Column():
                            content_text = gr.Textbox(
                                label="Content to Verify",
                                placeholder="Paste the content you want to fact-check...",
                                lines=8
                            )
                            source_url = gr.Textbox(
                                label="Source URL (Optional)",
                                placeholder="https://example.com/article"
                            )
                            verify_btn = gr.Button("🔍 Verify Content", variant="secondary")

                        with gr.Column():
                            verification_result = gr.Textbox(
                                label="AI Verification Analysis",
                                lines=12,
                                show_copy_button=True
                            )

                # Keep all other tabs unchanged...
                # Module 4: Admin Dashboard
                with gr.TabItem("📊 Admin Dashboard"):
                    gr.Markdown("### Real-time Analytics & Insights")

                    dashboard_btn = gr.Button("🔄 Refresh Dashboard", variant="primary")

                    with gr.Row():
                        query_volume_plot = gr.Plot(label="Query Volume Trends")
                        category_plot = gr.Plot(label="Category Distribution")

                    with gr.Row():
                        satisfaction_plot = gr.Plot(label="Satisfaction Scores")
                        language_plot = gr.Plot(label="Language Usage")

                # Module 5: Staff Portal
                with gr.TabItem("👨‍💼 Government Staff Portal"):
                    gr.Markdown("### Query Management & Resolution Tracking")

                    with gr.Row():
                        staff_role = gr.Dropdown(
                            choices=["Admin", "Manager", "Staff", "Specialist"],
                            label="Staff Role",
                            value="Staff"
                        )
                        filter_category = gr.Dropdown(
                            choices=["All"] + self.query_categories,
                            label="Filter by Category",
                            value="All"
                        )
                        filter_status = gr.Dropdown(
                            choices=["All", "Open", "In Progress", "Resolved", "Closed"],
                            label="Filter by Status",
                            value="All"
                        )
                        load_queries_btn = gr.Button("📋 Load Queries", variant="primary")

                    with gr.Row():
                        queries_table = gr.Dataframe(
                            label="Query Management Dashboard",
                            interactive=True
                        )
                        staff_summary = gr.Textbox(
                            label="Summary Statistics",
                            lines=10,
                            show_copy_button=True
                        )

                # Module 6: Analytics & Reporting
                with gr.TabItem("📈 Analytics & Reports"):
                    gr.Markdown("### Comprehensive Analytics & Predictive Insights")

                    with gr.Row():
                        report_type = gr.Radio(
                            choices=["Performance Metrics", "Trend Analysis", "Predictive Insights"],
                            label="Report Type",
                            value="Performance Metrics"
                        )
                        date_range = gr.Dropdown(
                            choices=["Last 7 days", "Last 30 days", "Last 3 months", "Last year"],
                            label="Date Range",
                            value="Last 30 days"
                        )
                        generate_report_btn = gr.Button("📊 Generate Report", variant="primary")

                    analytics_output = gr.Textbox(
                        label="Analytics Report",
                        lines=15,
                        show_copy_button=True
                    )

                # Module 7: Notifications
                with gr.TabItem("🔔 Notifications & Alerts"):
                    gr.Markdown("### Citizen Communication System")

                    with gr.Row():
                        with gr.Column():
                            notification_type = gr.Radio(
                                choices=["Service Update", "Emergency Alert", "Reminder", "Announcement"],
                                label="Notification Type",
                                value="Service Update"
                            )
                            target_audience = gr.CheckboxGroup(
                                choices=["All Citizens", "Specific Category", "Geographic Region", "Age Group"],
                                label="Target Audience",
                                value=["All Citizens"]
                            )
                            notification_message = gr.Textbox(
                                label="Message",
                                placeholder="Enter notification message...",
                                lines=4
                            )
                            channels = gr.CheckboxGroup(
                                choices=["SMS", "Email", "Push Notification", "Voice Call"],
                                label="Delivery Channels",
                                value=["SMS", "Push Notification"]
                            )
                            send_notification_btn = gr.Button("📤 Send Notification", variant="primary")

                        with gr.Column():
                            notification_result = gr.Textbox(
                                label="Delivery Status",
                                lines=12,
                                show_copy_button=True
                            )

                # Module 8: Authentication
                with gr.TabItem("🔐 Access Control"):
                    gr.Markdown("### Authentication & Role-based Access Management")

                    with gr.Row():
                        with gr.Column():
                            username = gr.Textbox(
                                label="Username",
                                placeholder="Enter username"
                            )
                            user_role = gr.Dropdown(
                                choices=["Citizen", "Staff", "Manager", "Admin"],
                                label="User Role",
                                value="Citizen"
                            )
                            access_level = gr.Radio(
                                choices=["Guest", "Basic", "Partial", "Full"],
                                label="Access Level",
                                value="Basic"
                            )
                            auth_btn = gr.Button("🔑 Authenticate", variant="primary")

                        with gr.Column():
                            auth_result = gr.Textbox(
                                label="Authentication Result",
                                lines=15,
                                show_copy_button=True
                            )

            # Event Handlers
            submit_query_btn.click(
                fn=self.process_citizen_query,
                inputs=[citizen_query, language_select, voice_input, query_category],
                outputs=query_response
            )

            chat_submit.click(
                fn=self.chatbot_response,
                inputs=[chat_input, chatbot],
                outputs=[chat_input, chatbot]
            )

            chat_input.submit(
                fn=self.chatbot_response,
                inputs=[chat_input, chatbot],
                outputs=[chat_input, chatbot]
            )

            verify_btn.click(
                fn=self.detect_misinformation,
                inputs=[content_text, source_url],
                outputs=verification_result
            )

            dashboard_btn.click(
                fn=self.generate_admin_dashboard,
                outputs=[query_volume_plot, category_plot, satisfaction_plot, language_plot]
            )

            load_queries_btn.click(
                fn=self.staff_query_management,
                inputs=[staff_role, filter_category, filter_status],
                outputs=[queries_table, staff_summary]
            )

            generate_report_btn.click(
                fn=self.generate_analytics_report,
                inputs=[report_type, date_range],
                outputs=analytics_output
            )

            send_notification_btn.click(
                fn=self.send_notifications,
                inputs=[notification_type, target_audience, notification_message, channels],
                outputs=notification_result
            )

            auth_btn.click(
                fn=self.authenticate_user,
                inputs=[username, user_role, access_level],
                outputs=auth_result
            )

            # Footer
            gr.Markdown(f"""
            ---
            **IBM Hackathon 2024** | Government Citizen Engagement Platform |
            Built with AI/ML, NLP, and Advanced Analytics

            **AI Status:** {'🟢 IBM Granite 3.3-2B Model Active' if self.model_loaded else '🟡 Fallback System Active'}
            """)

        return interface

def main():
    """Launch the Government Citizen Engagement Platform with IBM Granite AI"""
    print("🚀 Initializing Government Citizen Engagement Platform...")
    print("🔄 Loading IBM Granite 3.3-2B Instruct Model...")

    platform = GovernmentPlatform()
    interface = platform.create_interface()

    print("✅ Platform initialized successfully!")
    print(f"🤖 AI Model Status: {'IBM Granite 3.3-2B Active' if platform.model_loaded else 'Fallback System Active'}")

    interface.launch(
        server_name="0.0.0.0",
        server_port=7860,
        share=True,
        debug=True,
        show_error=True
    )

if __name__ == "__main__":
    main()

🚀 Initializing Government Citizen Engagement Platform...
🔄 Loading IBM Granite 3.3-2B Instruct Model...
Loading IBM Granite 3.3 2B Instruct model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.93k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-4-a2114dd63deb>:867: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


✅ IBM Granite model loaded successfully!
✅ Platform initialized successfully!
🤖 AI Model Status: IBM Granite 3.3-2B Active
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://742abc4bfe1e91671e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
